In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as dist

In [45]:
size1 = cv2.imread(r"E:\PyImage_ComputerVision\OpenCVBasic\WorkingData\InputData\Size1.png")
size2 = cv2.imread(r"E:\PyImage_ComputerVision\OpenCVBasic\WorkingData\InputData\Size2.png")
size3 = cv2.imread(r"E:\PyImage_ComputerVision\OpenCVBasic\WorkingData\InputData\Size3.png")

In [3]:
size1.shape

(415, 600, 3)

In [4]:
size2.shape

(600, 800, 3)

In [5]:
size3.shape

(450, 600, 3)

In [6]:
cv2.imshow("Size1",size1)
cv2.imshow("Size2",size2)
cv2.imshow("Size3",size3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return cnts, boundingBoxes

In [13]:
def order_points(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype=np.int32)

In [21]:
def midpoint(pt1,pt2):
    return int((pt1[0]+pt2[0])//2),int((pt1[1]+pt2[1])//2)

In [46]:
img = size1.copy()
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img,(7,7),0)
img = cv2.Canny(img,50,100)
img = cv2.dilate(img,None,iterations=1)
img = cv2.erode(img,None,iterations=1)
cnts = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
cnts,_ = sort_contours(cnts)
out_cn = size1.copy()
out_rc = size1.copy()
for c in cnts:
    if cv2.contourArea(c) < 100:
        continue
    box = cv2.minAreaRect(c)
    box = cv2.boxPoints(box)
    box = np.array(box,dtype=np.int16)
    box = order_points(box)
    for pt in box:
        print(pt)
        cv2.circle(out_cn,pt,2,(0,255,0),-1)
        cv2.circle(out_rc,pt,2,(0,255,0),-1)
        cv2.imshow("Out CN",out_cn)
        cv2.imshow("Out RC",out_rc)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    cv2.drawContours(out_cn,[box],-1,(255,0,0),2)
    cv2.rectangle(out_rc,box[0],box[2],(255,0,0),2)
    cv2.imshow("Out CN",out_cn)
    cv2.imshow("OUT RC",out_rc)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[ 42 190]
[ 90 190]
[ 90 236]
[ 42 236]
[171  94]
[342  59]
[362 157]
[192 192]
[348 183]
[395 183]
[395 229]
[348 229]
[377 258]
[489 237]
[512 363]
[400 383]
[450  42]
[553  92]
[495 211]
[392 161]
[520 197]
[549 226]
[520 255]
[491 226]


In [70]:
img = size1.copy()
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img,(7,7),0)
img = cv2.Canny(img,50,100)
img = cv2.dilate(img,None,iterations=1)
img = cv2.erode(img,None,iterations=1)
cnts = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
cnts,_ = sort_contours(cnts)
out = size1.copy()
PMMI = None
for c in cnts:
    if cv2.contourArea(c) < 100:
        continue
    box = cv2.minAreaRect(c)
    box = cv2.boxPoints(box)
    box = np.array(box,dtype="int")
    box = order_points(box)
    tl,tr,br,bl = box
    
    h1 = midpoint(tl,tr) #h1
    h2 = midpoint(bl,br) #h2

    w1 = midpoint(tl,bl) #w1
    w2 = midpoint(tr,br) #w2

    width = dist.euclidean(w1,w2)
    height = dist.euclidean(h1,h2)

    if PMMI is None:
        PMMI = width/0.955
        print(PMMI)
    
    width = width/PMMI
    height = height / PMMI
    print(width,height)
    '''cv2.putText(out,f"{round(width,2)}",(mdwt[0],mdwt[1]-10),0,1,(0,0,255))
    cv2.imshow("out",out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.putText(out,f"{round(height,2)}",(mdhr[0]+10,mdhr[1]),0,1,(0,0,255))
    cv2.imshow("out",out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

50.261780104712045
0.955 0.9152083333333333
3.462975384863071 1.9919942423964108
0.9351041666666666 0.9152083333333333
2.265352723701569 2.538512641850381
2.277964312367544 2.6338506649175852
0.8159717627067263 0.8159717627067263


In [69]:
# USAGE
# python object_size.py --image images/example_01.png --width 0.955
# python object_size.py --image images/example_02.png --width 0.955
# python object_size.py --image images/example_03.png --width 3.5

# import the necessary packages
from scipy.spatial import distance as dist

import numpy as np
import argparse
import cv2

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

image = size1.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)

# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0]

# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = sort_contours(cnts)
pixelsPerMetric = None

# loop over the contours individually
for c in cnts:
	# if the contour is not sufficiently large, ignore it
    if cv2.contourArea(c) < 100:
        continue

	# compute the rotated bounding box of the contour
    orig = image.copy()
    box = cv2.minAreaRect(c)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")

	# order the points in the contour such that they appear
	# in top-left, top-right, bottom-right, and bottom-left
	# order, then draw the outline of the rotated bounding
	# box
    box = order_points(box)
	# unpack the ordered bounding box, then compute the midpoint
	# between the top-left and top-right coordinates, followed by
	# the midpoint between bottom-left and bottom-right coordinates
    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr) #h1
    (blbrX, blbrY) = midpoint(bl, br) #h2

	# compute the midpoint between the top-left and top-right points,
	# followed by the midpoint between the top-righ and bottom-right
    (tlblX, tlblY) = midpoint(tl, bl) #w1
    (trbrX, trbrY) = midpoint(tr, br) #w2


	# compute the Euclidean distance between the midpoints
    dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY)) ##Hieght
    dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY)) ##Width
	# if the pixels per metric has not been initialized, then
	# compute it as the ratio of pixels to supplied metric
	# (in this case, inches)
    if pixelsPerMetric is None:
        pixelsPerMetric = dB / 0.955
        print(pixelsPerMetric)
	# compute the size of the object
    dimA = dA / pixelsPerMetric
    dimB = dB / pixelsPerMetric
    print(dimB,dimA)
# draw the object sizes on the image
'''cv2.putText(orig, "{:.1f}in".format(dimA),
		(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	cv2.putText(orig, "{:.1f}in".format(dimB),
		(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)

	# show the output image
	cv2.imshow("Image", orig)
	cv2.waitKey(0)'''

50.261780104712045
0.955 0.9152083333333333
3.462975384863071 1.9919942423964108
0.9351041666666666 0.9152083333333333
2.265352723701569 2.538512641850381
2.277964312367544 2.6338506649175852
0.8159717627067263 0.8159717627067263


'cv2.putText(orig, "{:.1f}in".format(dimA),\n\t\t(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,\n\t\t0.65, (255, 255, 255), 2)\n\tcv2.putText(orig, "{:.1f}in".format(dimB),\n\t\t(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,\n\t\t0.65, (255, 255, 255), 2)\n\n\t# show the output image\n\tcv2.imshow("Image", orig)\n\tcv2.waitKey(0)'